In [ ]:
!pip install psycopg2

In [2]:
import psycopg2
import io

In [13]:
import os
os.environ['PYTHONIOENCODING'] = 'utf-8'

In [ ]:
import locale
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

In [3]:
conn = psycopg2.connect(
    dbname="postgres",
    user="postgres",
    password="8483$Bia",
    host="localhost",
    port="5432"
)

In [4]:
cur = conn.cursor()

In [ ]:
with open('utils/build_script.sql', 'r') as file:
    sql_script = file.read()

cur.execute(sql_script)

In [5]:
cur.execute("SELECT * FROM distrito")

In [6]:
rows = cur.fetchall()
print(rows)

[('MANDAQUI', 'JACANA/TREMEMBE'), ('RAPOSO TAVARES', 'BUTANTA'), ('BRAS', 'PENHA'), ('CAMBUCI', 'IPIRANGA'), ('VILA ANDRADE', 'BUTANTA'), ('JOSE BONIFACIO', 'ITAQUERA'), ('VILA MATILDE', 'PENHA'), ('PONTE RASA', 'PENHA'), ('JAGUARA', 'PIRITUBA'), ('CIDADE LIDER', 'ITAQUERA'), ('IGUATEMI', 'SAO MATEUS'), ('CANGAIBA', 'PENHA'), ('VILA JACUI', 'SAO MIGUEL'), ('PENHA', 'PENHA'), ('VILA LEOPOLDINA', 'PIRITUBA'), ('LAPA', 'PIRITUBA'), ('TREMEMBE', 'JACANA/TREMEMBE'), ('VILA PRUDENTE', 'IPIRANGA'), ('CIDADE DUTRA', 'CAPELA DO SOCORRO'), ('VILA MEDEIROS', 'JACANA/TREMEMBE'), ('MOOCA', 'PENHA'), ('CARRAO', 'ITAQUERA'), ('JAGUARE', 'PIRITUBA'), ('PARELHEIROS', 'CAPELA DO SOCORRO'), ('CACHOEIRINHA', 'FREGUESIA/BRASILANDIA'), ('JARDIM ANGELA', 'CAMPO LIMPO'), ('VILA MARIA', 'JACANA/TREMEMBE'), ('CAMPO GRANDE', 'SANTO AMARO'), ('CAMPO LIMPO', 'CAMPO LIMPO'), ('JARDIM PAULISTA', 'BUTANTA'), ('SACOMA', 'IPIRANGA'), ('CIDADE ADEMAR', 'SANTO AMARO'), ('AGUA RASA', 'PENHA'), ('PIRITUBA', 'PIRITUBA'), ('

In [10]:
try:
    cur.execute("DROP TABLE IF EXISTS MATRICULA")
    cur.execute("""
        CREATE TABLE MATRICULA (
            AN_LETIVO                     numeric,
            CD_UNIDADE_EDUCACAO           numeric,
            NOME_DISTRITO                 text,
            CD_SETOR                      numeric,
            TIPO_ESCOLA                   text,
            NOME_ESCOLA                   text,
            DRE                           text,
            CD_INEP_ESCOLA                text,
            SITUACAO_ESCOLA               text,
            CD_TURMA                      numeric,
            CD_TURNO                      numeric,
            DESC_TURNO                    text,
            CD_SERIE                      numeric,
            DESC_SERIE                    text,
            MODALIDADE                    text,
            MODALIDADE_SEGMENTO           text,
            TURMA_ESCOL                   text,
            NOME_TURMA                    text,
            DESC_ETAPA_ENSINO             text,
            DESC_CICLO_ENSINO             text,
            DESC_TIPO_TURMA               text,
            CD_ALUNO_SME                  numeric,
            NASC_ALUNO                    timestamp,
            CD_SEXO                       text,
            DESC_RACA_COR                 text,
            DESC_PAIS_NASC                text,
            CD_MUN_NASC                   text,
            NEE                           text,
            CD_MAT                        numeric,
            DT_IN_MAT                     timestamp,
            DT_FIM_MAT                    timestamp,
            SITUACAO_MAT                  text,
            DATA_SIT                      timestamp
        )
    """)
    conn.commit()
except Exception as e:
    print("failed to create table:", e)
    conn.rollback()


In [16]:
try:
    cur.execute("DROP TABLE IF EXISTS MATRICULA")
    cur.execute("""
        CREATE TABLE MATRICULA (
            AN_LETIVO                     numeric,
            CD_UNIDADE_EDUCACAO           numeric,
            NOME_DISTRITO                 text,
            CD_SETOR                      numeric,
            TIPO_ESCOLA                   text,
            NOME_ESCOLA                   text,
            DRE                           text,
            CD_INEP_ESCOLA                text,
            CD_TURNO                      numeric,
            DESC_TURNO                    text,
            CD_SERIE                      numeric,
            DESC_SERIE                    text,
            MODALIDADE                    text,
            NOME_TURMA                    text,
            DESC_ETAPA_ENSINO             text,
            DESC_CICLO_ENSINO             text,
            DESC_TIPO_TURMA               text,
            CD_ALUNO_SME                  numeric,
            NASC_ALUNO                    timestamp,
            CD_SEXO                       text,
            DESC_RACA_COR                 text,
            DESC_PAIS_NASC                text,
            NEE                           text,
            CD_MAT                        numeric,
            DT_IN_MAT                     timestamp,
            DT_FIM_MAT                    timestamp,
            SITUACAO_MAT                  text,
            DATA_SIT                      timestamp
        )
    """)
    conn.commit()
except Exception as e:
    print("failed to create table:", e)
    conn.rollback()


In [ ]:
copy_command = """
COPY MATRICULA (
    AN_LETIVO,
    CD_UNIDADE_EDUCACAO,
    NOME_DISTRITO,
    CD_SETOR,
    TIPO_ESCOLA,
    NOME_ESCOLA,
    DRE,
    CD_INEP_ESCOLA,
    CD_TURNO,
    DESC_TURNO,
    CD_SERIE,
    DESC_SERIE,
    MODALIDADE,
    NOME_TURMA,
    DESC_ETAPA_ENSINO,
    DESC_CICLO_ENSINO,
    DESC_TIPO_TURMA,
    CD_ALUNO_SME,
    NASC_ALUNO,
    CD_SEXO,
    DESC_RACA_COR,
    DESC_PAIS_NASC,
    NEE,
    CD_MAT,
    DT_IN_MAT,
    DT_FIM_MAT,
    SITUACAO_MAT,
    DATA_SIT
)
FROM STDIN WITH (
    FORMAT CSV,
    HEADER,
    DELIMITER ';',
    NULL ''
);
"""

try:

    for file in os.listdir('datasets'):
        if file.endswith('.csv') and 'Microdados' in file:
            with open(os.path.join('datasets', file), 'r', encoding='utf-8-sig') as f:
                class CommaToDot(io.TextIOBase):
                    def __init__(self, f):  self.f = f
                    def read(self, size=-1):
                        chunk = self.f.read(size)
                        return chunk.replace(',', '.') if chunk else ''

                fixer = CommaToDot(f)
                cur.copy_expert(copy_command, fixer)
                conn.commit()

except Exception as e:
    print("failed to create table:", e)
    conn.rollback()

InFailedSqlTransaction: ERRO:  transação atual foi interrompida, comandos ignorados até o fim do bloco de transação


In [ ]:
cur.execute("""
DROP TABLE IF EXISTS TURMA;

CREATE TABLE TURMA (
    DRE               TEXT,
    CODINEP           NUMERIC,
    TIPOESC           TEXT,
    NOMESC            TEXT,
    SUBPREF           TEXT,
    DISTRITO          TEXT,
    CODAMB            NUMERIC,
    DESCAMB           TEXT,
    CAPREAL           NUMERIC,
    METRAGEM          NUMERIC,
    MODAL             TEXT,
    REDE              TEXT,
    CODSERIE          NUMERIC,
    DESCSERIE         TEXT,
    TURNO             NUMERIC,
    TURMA             TEXT,
    VAGOFER           NUMERIC,
    MATRIC            NUMERIC
)
""")
conn.commit()

In [ ]:
copy_command = """
COPY TURMA (
    DRE,
    CODINEP,
    TIPOESC,
    NOMESC,
    SUBPREF,
    DISTRITO,
    CODAMB,
    DESCAMB,
    CAPREAL,
    METRAGEM,
    MODAL,
    REDE,
    CODSERIE,
    DESCSERIE,
    TURNO,
    TURMA,
    VAGOFER,
    MATRIC
)
FROM STDIN WITH (
    FORMAT CSV,
    HEADER,
    DELIMITER ';',
    NULL ''
);
"""

with open('datasets/turmas_processed.csv', 'r', encoding='utf-8-sig') as f:
    cur.copy_expert(copy_command, f)
    conn.commit()

In [ ]:
cur.execute("""
DROP TABLE IF EXISTS PARCERIA;

CREATE TABLE PARCERIA (
    NUM_PROTOCOLO       NUMERIC,
    DRE                 TEXT,
    OSC_PARCEIRA        TEXT,
    CNPJ                TEXT,
    CODIGO_ESCOLA       NUMERIC,
    UNIDADE_EDUCACIONAL TEXT,
    VALOR_MENSAL        NUMERIC,
    VERBA_LOCACAO       NUMERIC,
    VALOR_MENSAL_IPTU   NUMERIC,
    DATA_INICIO         TIMESTAMP,
    DATA_TERMINO        TIMESTAMP
)
""")
conn.commit()

In [ ]:
copy_command = """
COPY PARCERIA (
    NUM_PROTOCOLO,
    DRE,
    OSC_PARCEIRA,
    CNPJ,
    CODIGO_ESCOLA,
    UNIDADE_EDUCACIONAL,
    VALOR_MENSAL,
    VERBA_LOCACAO,
    VALOR_MENSAL_IPTU,
    DATA_INICIO,
    DATA_TERMINO
)
FROM STDIN WITH (
    FORMAT CSV,
    HEADER,
    DELIMITER ';',
    NULL ''
);
"""

with open('datasets/parcerias_processed.csv', 'r', encoding='utf-8-sig') as f:
    cur.copy_expert(copy_command, f)
    conn.commit()

In [ ]:
conn.commit()
cur.close()
conn.close()